In [1]:
import numpy as np
import pandas as pd
from src.data.load import load_dataset
from src.data.process import extract_targets

data = load_dataset()
for k, v in data.items():
    print(len(v))

1044681
184937
178297
37960


提取特征

In [3]:
from src.data.features import extract_user_feature, extract_features

user_features = extract_user_feature(data['train'], data['all_uid'])
features_train = extract_features(data['train'], user_features)
features_valid = extract_features(data['valid'], user_features)
features_test = extract_features(data['test'], user_features)
print('feature dim:', features_train.shape[1])
targets_train = extract_targets(data['train'], 'log')
targets_valid = extract_targets(data['valid'], 'linear')

Counting User Histories: 100%|██████████| 1044681/1044681 [00:20<00:00, 52209.84it/s]


feature dim: 34


线性回归 - 拟合+预测

In [20]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from scipy.stats import randint, uniform

# 构建模型
model = xgb.XGBRegressor()

# 定义参数分布s
param_dist = {
    'max_depth': randint(3, 16),
    'learning_rate': uniform(0.001, 0.1),
    'n_estimators': randint(16, 256),
    'min_child_weight': randint(1, 10)
}

# 执行随机搜索
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=20, cv=3, n_jobs=-1)
random_result = random_search.fit(features_train, targets_train)

# 输出最佳参数组合和得分
print("Best Parameters: ", random_result.best_params_)
print("Best Score: ", random_result.best_score_)

Best Parameters:  {'learning_rate': 0.09445274248684067, 'max_depth': 8, 'min_child_weight': 6, 'n_estimators': 131}
Best Score:  0.5847875778701105


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
model = RandomForestRegressor(n_estimators=50, n_jobs=-1)

# 定义参数网格
param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

# 执行网格搜索
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_result = grid_search.fit(features_train, targets_train)

# 输出最佳参数组合和得分
print("Best Parameters: ", grid_result.best_params_)
print("Best Score: ", grid_result.best_score_)

C:\Users\wzq\anaconda3\envs\weibo\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
81 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
34 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\wzq\anaconda3\envs\weibo\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\wzq\anaconda3\envs\weibo\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\wzq\anaconda3\envs\weibo\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Use

Best Parameters:  {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10}
Best Score:  0.5842349968803543


In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
# model = LinearRegression()
model = RandomForestRegressor(n_estimators=50, max_depth=20, min_samples_leaf=4, min_samples_split=10, max_features='sqrt' , n_jobs=-1)
# model = xgb.XGBRegressor(max_depth=8, learning_rate=0.09445274248684067, n_estimators=131, min_child_weight=6, n_jobs=-1)
model.fit(features_train, targets_train)
predicts = model.predict(features_valid)
predicts.shape

(184937, 3)

In [28]:
from src.metric import compute_metrics

# 计算指标时恢复对数
def exp_interactions(x: np.ndarray) -> np.ndarray:
    """先计算指数，然后取整"""
    return np.rint(np.exp(x) - 1).astype(int)
compute_metrics(exp_interactions(predicts), targets_valid)

{'score': 0.352412611246109}

测试集结果提交

In [29]:
test_set_predicts = exp_interactions(model.predict(features_test))
results = []
for i in range(len(data['test'])):
    results.append(
        "{}\t{}\t{},{},{}\n".format(
            data['test'].loc[i, 'uid'],
            data['test'].loc[i, 'mid'],
            test_set_predicts[i, 1],
            test_set_predicts[i, 2],
            test_set_predicts[i, 0]
        )
    )
with open("submission.txt", 'w') as f:
    f.writelines(results)